# RNN pour l'analyse de sentiments

Nous allons maintenant faire une analyse de sentiments sur le même jeu de données en utilisant les réseaux de neurones récurrents

## Réseaux de neurones récurrents

Les réseaux de neurones récurrents ou (RNN), sont souvent utilisés pour analyser des séquences.

En effet, dans les réseaux de neurones généraux, un input est traité par un certain nombre de couches et un output est produit à la sortie, avec l'hypothèse que deux inputs successifs sont indépendants.

Cependant, cette hypothèse n'est pas correcte dans un certain nombre de scénarios. 
Par exemple, si on veut prédire le mot suivant dans une séquence, il est indispensable de considérer la dépendance des observations précédentes.
    
Dans notre cas, le modèle RNN prend une séquence de mots $X=\{x_1, ..., x_T\}$, une à la fois, et produit un état caché $h$, pour chaque mot.
On utilise le RNN en lui donnant le mot courant $x_t$ ainsi que l'état caché du mot précédent, $h_{t-1}$, pour produire l'état caché suivant, $h_t$.


$$h_t = \text{RNN}(x_t, h_{t-1})$$


Une fois que l'on a notre état caché final, $h_T$, obtenu après avoir donné le dernier mot de la séquence $x_T$ au modèle, on le donne à une couche linéaire $f$, (qui s'appelle également fully connected layer), pour recevoir notre sentiment prédit, $\hat{y} = f(h_T)$.

<center> <img src="RNN.png" alt="drawing" width="700"/>
        
        
Cette illustration montre un exemple de phrase, avec le RNN prédisant 0, c'est-à-dire que le sentiment est négatif. Le RNN est représenté en orange et la couche linéaire est en gris. On utilise le même RNN pour chaque mot, c'est-à-dire qu'il a les mêmes paramètres. L'état initial caché $h_0$, est un tensor initialisé à zéro.

## Préparation des données

Dans ce notebook, on utilise la librairie torch et TorchText.

TorchText a une méthode `Field` qui sert à définir comment les données brutes doivent être traitées.

La méthode `TEXT` définit comment les commentaires doivent être traités, et `LABEL` comment les labels doivent être traités. 

`TEXT` a un argument `tokenize`, qui par défaut split les chaînes de caractères en espaces.

In [1]:
import pandas as pd
import torch
from torchtext import data

SEED = 1234
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(sequential=True,lower=True,include_lengths = True)
LABEL = data.LabelField(dtype = torch.float)

On sépare les données en train et test.

In [2]:
train_data, valid_data, test_data = data.TabularDataset.splits(
        path='./data/', train='train.csv',
        validation='valid.csv', test='test.csv', format='csv', skip_header=True,
        fields=[('text', TEXT), ('label', LABEL)])

print(f'Taille des données train: {len(train_data)}')
print(f'Taille des données de validation: {len(valid_data)}')
print(f'Taille des données test: {len(test_data)}')

Taille des données train: 18163
Taille des données de validation: 2270
Taille des données test: 2271


On affiche un exemple.

In [3]:
print(vars(train_data.examples[0]))

{'text': ['this', 'movie', 'is', 'a', 'gem', 'because', 'it', 'moves', 'with', 'soft,', 'but', 'firm', 'resolution.<br', '/><br', '/>i', 'caution', 'viewers', 'that', 'although', 'it', 'is', 'billed', 'as', 'a', 'corporate', 'spy', 'thriller', 'and', 'ms', 'liu', 'is', 'there,', 'it', 'moves', 'at', 'a', 'deftly', 'purposeful', 'yet', 'sedate', 'pace.', "it's", 'not', 'about', 'explosions,', 'car', 'chases,', 'or', 'flying', 'bullets.', 'you', 'must', 'be', 'patient', 'and', 'instead,', 'note', 'the', 'details', 'here.', "it's", 'sedate', 'because', "that's", 'what', 'the', 'main', 'character', 'is.', 'the', 'viewer', 'has', 'to', 'watch', 'him', 'and', 'think', 'as', 'this', 'story', 'unfolds.<br', '/><br', '/>i', 'will', 'not', 'give', 'spoilers--', 'because', 'that', 'destroys', 'the', 'point', 'of', 'watching.', 'the', 'plot', 'is', 'what', "you've", 'read', 'from', 'the', 'other', 'postings:', 'an', 'average', 'white-collar', 'guy,', 'seeking', 'change', 'and', 'adventure,', 'sign

On crée un échantillon de données de validation.

On crée un vocabulaire sur l'échantillon train avec la méthode `build_vocab`et on ne prend que 25000 mots pour diminuer la dimension de la matrice d'embedding.

In [4]:
MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train_data, max_size = MAX_VOCAB_SIZE)
LABEL.build_vocab(train_data)

print(f"Nombre de tokens unique dans le TEXT: {len(TEXT.vocab)}") 
print(f"Nombre unique de LABEL: {len(LABEL.vocab)}")

Les deux valeurs supplémentaires dans le vocabulaire du TEXT sont les tokens `<unk>` et `<pad>`.


In [6]:
print(TEXT.vocab.freqs.most_common(20))

[('the', 269481), ('a', 134161), ('and', 132447), ('of', 121047), ('to', 111764), ('is', 88203), ('in', 76185), (',', 55882), ('that', 55193), ('this', 53162), ('it', 52750), ('i', 51962), ('.', 46745), ('as', 38794), ('with', 36382), ('for', 35734), ('was', 34423), ('/><br', 33836), ('but', 32810), ('his', 26298)]


We can also check the labels, ensuring 0 is for negative and 1 is for positive.

In [8]:
print(LABEL.vocab.stoi)

defaultdict(None, {'0': 0, '1': 1})


La dernière étape de la préparation des données consiste à créer les itérateurs. Nous les parcourons dans la boucle d'apprentissage / d'évaluation, et ils retournent un lot d'exemples (indexés et convertis en tenseurs) à chaque itération.

On utilise `BucketIterator` qui est un itérateur qui renverra un lot d'exemples où chaque exemple est d'une longueur similaire, minimisant la quantité de padding par exemple.

In [9]:
# utilisation du GPU si possible 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

BATCH_SIZE = 64
train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    device = device, sort = False)

In [25]:
for tab in test_iterator:
    print(tab)


[torchtext.data.batch.Batch of size 64]
	[.text]:('[torch.cuda.LongTensor of size 2678x64 (GPU 0)]', '[torch.cuda.LongTensor of size 64 (GPU 0)]')
	[.label]:[torch.cuda.FloatTensor of size 64 (GPU 0)]

[torchtext.data.batch.Batch of size 64]
	[.text]:('[torch.cuda.LongTensor of size 1263x64 (GPU 0)]', '[torch.cuda.LongTensor of size 64 (GPU 0)]')
	[.label]:[torch.cuda.FloatTensor of size 64 (GPU 0)]

[torchtext.data.batch.Batch of size 64]
	[.text]:('[torch.cuda.LongTensor of size 927x64 (GPU 0)]', '[torch.cuda.LongTensor of size 64 (GPU 0)]')
	[.label]:[torch.cuda.FloatTensor of size 64 (GPU 0)]

[torchtext.data.batch.Batch of size 64]
	[.text]:('[torch.cuda.LongTensor of size 1090x64 (GPU 0)]', '[torch.cuda.LongTensor of size 64 (GPU 0)]')
	[.label]:[torch.cuda.FloatTensor of size 64 (GPU 0)]

[torchtext.data.batch.Batch of size 64]
	[.text]:('[torch.cuda.LongTensor of size 1187x64 (GPU 0)]', '[torch.cuda.LongTensor of size 64 (GPU 0)]')
	[.label]:[torch.cuda.FloatTensor of size 64 

## Construction du modèle

Le modèle RNN utilisé se compose des couches suivantes : 

- _embedding_ : utilisé pour transformer notre vecteur one hot encoder (dont la plupart des éléments sont des 0) en un vecteur embedding dense (dense car la dimensionnalité est beaucoup plus petite et tous les éléments sont des nombres réels). De plus, les mots qui ont un impact similaire sur le sentiment de la revue sont mappés de manière rapprochée dans cet espace vectoriel dense.
- _RNN_ : prend le vecteur dense et le précédent état caché $h_{t-1}$, et calcule l'état caché suivant $h_t$.
- _linéaire_ : prend le dernier état caché, le met dans un couche fully connected $f(h_T)$ qui le transfome en output prédit.

On appelle la méthode `forward` lorsque l'on donne nos exemple à notre modèle.

Chaque batch, `text` est un tensor de dimension _**[sentence length, batch size]**_. C'est un batch de commentaires, 

Each batch, , is a tensor of size . That is a batch of sentences, each having each word converted into a one-hot vector. 

You may notice that this tensor should have another dimension due to the one-hot vectors, however PyTorch conveniently stores a one-hot vector as it's index value, i.e. the tensor representing a sentence is just a tensor of the indexes for each token in that sentence. The act of converting a list of tokens into a list of indexes is commonly called *numericalizing*.

The input batch is then passed through the embedding layer to get `embedded`, which gives us a dense vector representation of our sentences. `embedded` is a tensor of size _**[sentence length, batch size, embedding dim]**_.

`embedded` is then fed into the RNN. In some frameworks you must feed the initial hidden state, $h_0$, into the RNN, however in PyTorch, if no initial hidden state is passed as an argument it defaults to a tensor of all zeros.

The RNN returns 2 tensors, `output` of size _**[sentence length, batch size, hidden dim]**_ and `hidden` of size _**[1, batch size, hidden dim]**_. `output` is the concatenation of the hidden state from every time step, whereas `hidden` is simply the final hidden state. We verify this using the `assert` statement. Note the `squeeze` method, which is used to remove a dimension of size 1. 

Finally, we feed the last hidden state, `hidden`, through the linear layer, `fc`, to produce a prediction.

In [10]:


import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        
        super().__init__()
        
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        
        self.rnn = nn.RNN(embedding_dim, hidden_dim)
        
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text):

        embedded = self.embedding(text)
        
        #embedded = [sent len, batch size, emb dim]
        
        output, hidden = self.rnn(embedded)
        
        #output = [sent len, batch size, hid dim]
        #hidden = [1, batch size, hid dim]
        
        assert torch.equal(output[-1,:,:], hidden.squeeze(0))
        
        return self.fc(hidden.squeeze(0))



We now create an instance of our RNN class. 

The input dimension is the dimension of the one-hot vectors, which is equal to the vocabulary size. 

The embedding dimension is the size of the dense word vectors. This is usually around 50-250 dimensions, but depends on the size of the vocabulary.

The hidden dimension is the size of the hidden states. This is usually around 100-500 dimensions, but also depends on factors such as on the vocabulary size, the size of the dense vectors and the complexity of the task.

The output dimension is usually the number of classes, however in the case of only 2 classes the output value is between 0 and 1 and thus can be 1-dimensional, i.e. a single scalar real number.

In [12]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1

model = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)

Let's also create a function that will tell us how many trainable parameters our model has so we can compare the number of parameters across different models.

In [13]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 2,592,105 trainable parameters


## Train the Model

Now we'll set up the training and then train the model.

First, we'll create an optimizer. This is the algorithm we use to update the parameters of the module. Here, we'll use _stochastic gradient descent_ (SGD). The first argument is the parameters will be updated by the optimizer, the second is the learning rate, i.e. how much we'll change the parameters by when we do a parameter update.

In [14]:
import torch.optim as optim

optimizer = optim.SGD(model.parameters(), lr=1e-3)

Next, we'll define our loss function. In PyTorch this is commonly called a criterion. 

The loss function here is _binary cross entropy with logits_. 

Our model currently outputs an unbound real number. As our labels are either 0 or 1, we want to restrict the predictions to a number between 0 and 1. We do this using the _sigmoid_ or _logit_ functions. 

We then use this this bound scalar to calculate the loss using binary cross entropy. 

The `BCEWithLogitsLoss` criterion carries out both the sigmoid and the binary cross entropy steps.

In [15]:
criterion = nn.BCEWithLogitsLoss()

Using `.to`, we can place the model and the criterion on the GPU (if we have one). 

In [16]:
model = model.to(device)
criterion = criterion.to(device)

Our criterion function calculates the loss, however we have to write our function to calculate the accuracy. 

This function first feeds the predictions through a sigmoid layer, squashing the values between 0 and 1, we then round them to the nearest integer. This rounds any value greater than 0.5 to 1 (a positive sentiment) and the rest to 0 (a negative sentiment).

We then calculate how many rounded predictions equal the actual labels and average it across the batch.

In [17]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

The `train` function iterates over all examples, one batch at a time. 

`model.train()` is used to put the model in "training mode", which turns on _dropout_ and _batch normalization_. Although we aren't using them in this model, it's good practice to include it.

For each batch, we first zero the gradients. Each parameter in a model has a `grad` attribute which stores the gradient calculated by the `criterion`. PyTorch does not automatically remove (or "zero") the gradients calculated from the last gradient calculation, so they must be manually zeroed.

We then feed the batch of sentences, `batch.text`, into the model. Note, you do not need to do `model.forward(batch.text)`, simply calling the model works. The `squeeze` is needed as the predictions are initially size _**[batch size, 1]**_, and we need to remove the dimension of size 1 as PyTorch expects the predictions input to our criterion function to be of size _**[batch size]**_.

The loss and accuracy are then calculated using our predictions and the labels, `batch.label`, with the loss being averaged over all examples in the batch.

We calculate the gradient of each parameter with `loss.backward()`, and then update the parameters using the gradients and optimizer algorithm with `optimizer.step()`.

The loss and accuracy is accumulated across the epoch, the `.item()` method is used to extract a scalar from a tensor which only contains a single value.

Finally, we return the loss and accuracy, averaged across the epoch. The `len` of an iterator is the number of batches in the iterator.

You may recall when initializing the `LABEL` field, we set `dtype=torch.float`. This is because TorchText sets tensors to be `LongTensor`s by default, however our criterion expects both inputs to be `FloatTensor`s. Setting the `dtype` to be `torch.float`, did this for us. The alternative method of doing this would be to do the conversion inside the `train` function by passing `batch.label.float()` instad of `batch.label` to the criterion. 

In [18]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
               
        predictions = model(batch.text).squeeze(1)
        print('predictions',predictions.size())
        loss = criterion(predictions, batch.label)
        print('loss',loss.size())
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

`evaluate` is similar to `train`, with a few modifications as you don't want to update the parameters when evaluating.

`model.eval()` puts the model in "evaluation mode", this turns off _dropout_ and _batch normalization_. Again, we are not using them in this model, but it is good practice to include them.

No gradients are calculated on PyTorch operations inside the `with no_grad()` block. This causes less memory to be used and speeds up computation.

The rest of the function is the same as `train`, with the removal of `optimizer.zero_grad()`, `loss.backward()` and `optimizer.step()`, as we do not update the model's parameters when evaluating.

In [19]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model.forward(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

We'll also create a function to tell us how long an epoch takes to compare training times between models.


In [20]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

We then train the model through multiple epochs, an epoch being a complete pass through all examples in the training and validation sets.

At each epoch, if the validation loss is the best we have seen so far, we'll save the parameters of the model and then after training has finished we'll use that model on the test set.

In [21]:
N_EPOCHS = 3

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

TypeError: embedding(): argument 'indices' (position 2) must be Tensor, not tuple

You may have noticed the loss is not really decreasing and the accuracy is poor. This is due to several issues with the model which we'll improve in the next notebook.

Finally, the metric we actually care about, the test loss and accuracy, which we get from our parameters that gave us the best validation loss.

In [ ]:
model.load_state_dict(torch.load('tut1-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

In [ ]:
import spacy
nlp = spacy.load('en')

def predict_sentiment(model, sentence):
    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    print(tokenized)
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    print(indexed)
    length = [len(indexed)]
    print(length)
    tensor = torch.LongTensor(indexed).to(device)
    print(tensor)
    tensor = tensor.unsqueeze(1)
    print(tensor)
    length_tensor = torch.LongTensor(length)
    print(length_tensor)
    prediction = torch.sigmoid(model(tensor, length_tensor))
    print(prediction)
    return prediction.item()

In [ ]:
predict_sentiment(model, "This film is terrible")

In [ ]:
predict_sentiment(model, "This film is great")